>Instituto Federal de Educação, Ciência e Tecnologia
>>Câmpus Campinas<br>
>>D3TOP – Tópicos em Ciência de Dados<br>
>>Prof.: Samuel Botter Martins<br>

## Preprocessing and data clean step

### Environment Config

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
%%capture
!pip install -U spacy
!pip install -U gensim
!pip install -U neattext
!pip install -U pyspellchecker
!pip install -U stanza 
!pip install unidecode

In [4]:
# Importando bibliotecas e setando parâmetros

import numpy as np
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# seta o idioma principal do dataset a ser processado e limpo
LANG = 'english' # or 'portuguese'

#### Useful Functions

In [5]:
def remove_cols(df, mask_cmds, inverse=True):
  '''
  Remove colunas de um dado dataframe df, a partir de uma máscara booleana mask.

  Parameters:
    df (pd.Dataframe): dataframe no qual sofrera a operacao de remocao de colunas
    mask_cmds (list): array de expressões a serem avaliadas. As expressões deve retornar array de booleanos contendo True nas posições das colunas que se deseja remover
    inverse (bool): (opcional) - indica se a máscara será invertida ou não. Default: True

  Returns:
    df (pd.Dataframe): dataframe sem as colunas indicadas na máscara
  '''
  mask_exp = mask_cmds.pop(0)
  mask = eval(mask_exp)
  if inverse:
    return remove_cols(df[df.columns[~mask]], mask_cmds) if len(mask_cmds) else df[df.columns[~mask]]
  return remove_cols(df[df.columns[mask]], mask_cmds) if len(mask_cmds) else df[df.columns[mask]]

In [6]:
def missing_values_table(df):
  '''
  Funcao que retorna o calculo de valores ausentes, absoluto e percentual, das colunas de um dado dataframe pandas

  Parameters:
    df (pd.Dataframe): dataframe que será  avaliado

  Returns:
    df (pd.Dataframe): dataframe contendo nas linhas o nome das colunas do dataframe original, e nas colunas
    a quantidade total de valores ausentes e o seu percentual correspondente.
  '''
  # Total missing values
  mis_val = df.isnull().sum()
  
  # Percentage of missing values
  mis_val_percent = 100 * df.isnull().sum() / len(df)
  
  # Make a table with the results
  mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
  
  # Rename the columns
  mis_val_table_ren_columns = mis_val_table.rename(
  columns = {0 : 'Missing Values', 1 : '% of Total Values'})
  
  # Sort the table by percentage of missing descending
  mis_val_table_ren_columns = mis_val_table_ren_columns[
      mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
  '% of Total Values', ascending=False).round(1)
  
  # Print some summary information
  print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
      "There are " + str(mis_val_table_ren_columns.shape[0]) +
        " columns that have missing values.")
  
  # Return the dataframe with missing information
  return mis_val_table_ren_columns

In [7]:
def neat_preprocessing(text_in: str ) -> str:
  '''
  Funcao auxiliar que executa algumas etapas de limpeza em um dado texto usando a biblioteca neattext.
  - Remove tags HTML, URLs, pontuacoes, emails, numeros, simbolos de moeda, espaços multiplos
  - Substitue @usernames pela palavra "USER"
  - Substitue codigos de emogis pela palavra "EMOJI"

  Parameters:
    text_in (str): string de texto que sofrera o processo de limpeza.

  Returns:
    text: string de texto apos o processo de limpeza.
  '''
  import neattext.functions as ntx

  text = text_in.lower()
  
  # text = ntx.fix_contractions(text)
  text = ntx.remove_html_tags(text)
  text = ntx.remove_punctuations(text) 
  text = ntx.replace_term(text, ntx.USER_HANDLES_REGEX, 'USER')  
  text = ntx.remove_urls(text)
  text = ntx.remove_emails(text)
  text = ntx.remove_phone_numbers(text)
  text = ntx.remove_numbers(text)  
  text = ntx.replace_emojis(text, "EMOJI")
  text = ntx.remove_multiple_spaces(text)
  text = ntx.remove_currency_symbols(text)
  
  return text


In [8]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

tqdm.pandas()

def parallel_applymap(df, func, worker_count, **kwargs):
  '''
  Funcao que paraleliza o metodo applymap do pandas

  Parameters:
    df (pd.DataFrame): dataframe no qual sera aplicado a funcao.
    func (callable):  funcao a ser aplicada a todas a celulas do dataframe.
    worker_count (int): quantidade de tarefas em paralelo que serão iniciadas. O dataframe sera subdividido em um numero = worker_count para cada tarefa processar.
    kwargs: demais parametros da funcao

  Returns:
    pd.Dataframe concatenado.
  '''
  def _apply(shard):
      return shard.progress_applymap(func, **kwargs)

  shards = np.array_split(df, worker_count)
  with ThreadPoolExecutor(max_workers=worker_count) as e:
      futures = e.map(_apply, shards)
  return pd.concat(list(futures))

### Data gathering

The data gathering procedure was done through [Apify](https://apify.com/dtrungtin/airbnb-scraper/input-schema#locationQuery), a web scraping and automation platform.

In [9]:
%cd /content/drive/MyDrive/Colab Notebooks/IFSP/Trabalhos_Grupo_IFSP/NLP

/content/drive/MyDrive/Colab Notebooks/IFSP/Trabalhos_Grupo_IFSP/NLP


In [10]:
file_path = 'data/raw/'
file_name = 'dataset_airbnb-scraper_2023-04-04_01-45-49-997.csv'

df_raw = pd.read_csv(file_path + file_name, dtype=str)
print(f'No Rows: {df_raw.shape[0]}')
print(f'No Columns: {df_raw.shape[1]}', end='\n\n')
display(df_raw.head())

No Rows: 547
No Columns: 1107



,additionalHosts/0/about,additionalHosts/0/badges/0,additionalHosts/0/firstName,additionalHosts/0/hasInclusionBadge,additionalHosts/0/hasProfilePic,additionalHosts/0/hostUrl,additionalHosts/0/id,additionalHosts/0/isSuperHost,additionalHosts/0/languages/0,additionalHosts/0/languages/1,additionalHosts/0/languages/2,additionalHosts/0/languages/3,additionalHosts/0/languages/4,additionalHosts/0/memberSince,additionalHosts/0/pictureUrl,additionalHosts/0/responseRate,additionalHosts/0/responseTime,additionalHosts/0/smartName,additionalHosts/0/thumbnailUrl,additionalHosts/1/about,additionalHosts/1/badges/0,additionalHosts/1/firstName,additionalHosts/1/hasInclusionBadge,additionalHosts/1/hasProfilePic,additionalHosts/1/hostUrl,additionalHosts/1/id,additionalHosts/1/isSuperHost,additionalHosts/1/languages/0,additionalHosts/1/languages/1,additionalHosts/1/memberSince,additionalHosts/1/pictureUrl,additionalHosts/1/responseRate,additionalHosts/1/responseTime,additionalHosts/1/smartName,additionalHosts/1/thumbnailUrl,additionalHosts/2/about,additionalHosts/2/firstName,additionalHosts/2/hasInclusionBadge,additionalHosts/2/hasProfilePic,additionalHosts/2/hostUrl,additionalHosts/2/id,additionalHosts/2/isSuperHost,additionalHosts/2/memberSince,additionalHosts/2/pictureUrl,additionalHosts/2/smartName,additionalHosts/2/thumbnailUrl,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,photos/1/caption,photos/1/pictureUrl,photos/1/thumbnailUrl,photos/2/caption,photos/2/pictureUrl,photos/2/thumbnailUrl,photos/3/caption,photos/3/pictureUrl,photos/3/thumbnailUrl,photos/4/caption,photos/4/pictureUrl,photos/4/thumbnailUrl,photos/5/caption,photos/5/pictureUrl,photos/5/thumbnailUrl,photos/6/caption,photos/6/pictureUrl,photos/6/thumbnailUrl,photos/7/caption,photos/7/pictureUrl,photos/7/thumbnailUrl,photos/8/caption,photos/8/pictureUrl,photos/8/thumbnailUrl,photos/9/caption,photos/9/pictureUrl,photos/9/thumbnailUrl,photos/10/caption,photos/10/pictureUrl,photos/10/thumbnailUrl,photos/11/caption,photos/11/pictureUrl,photos/11/thumbnailUrl,photos/12/caption,photos/12/pictureUrl,photos/12/thumbnailUrl,photos/13/caption,photos/13/pictureUrl,photos/13/thumbnailUrl,photos/14/caption,photos/14/pictureUrl,photos/14/thumbnailUrl,photos/15/caption,photos/15/pictureUrl,photos/15/thumbnailUrl,photos/16/caption,photos/16/pictureUrl,photos/16/thumbnailUrl,photos/17/caption,photos/17/pictureUrl,photos/17/thumbnailUrl,photos/18/caption,photos/18/pictureUrl,photos/18/thumbnailUrl,photos/19/caption,photos/19/pictureUrl,photos/19/thumbnailUrl,photos/20/caption,photos/20/pictureUrl,photos/20/thumbnailUrl,photos/21/caption,photos/21/pictureUrl,photos/21/thumbnailUrl,photos/22/caption,photos/22/pictureUrl,photos/22/thumbnailUrl,photos/23/caption,photos/23/pictureUrl,photos/23/thumbnailUrl,photos/24/caption,photos/24/pictureUrl,photos/24/thumbnailUrl,photos/25/caption,photos/25/pictureUrl,photos/25/thumbnailUrl,photos/26/caption,photos/26/pictureUrl,photos/26/thumbnailUrl,photos/27/caption,photos/27/pictureUrl,photos/27/thumbnailUrl,photos/28/caption,photos/28/pictureUrl,photos/28/thumbnailUrl,photos/29/caption,photos/29/pictureUrl,photos/29/thumbnailUrl,photos/30/caption,photos/30/pictureUrl,photos/30/thumbnailUrl,photos/31/caption,photos/31/pictureUrl,photos/31/thumbnailUrl,photos/32/caption,photos/32/pictureUrl,photos/32/thumbnailUrl,photos/33/caption,photos/33/pictureUrl,photos/33/thumbnailUrl,photos/34/caption,photos/34/pictureUrl,photos/34/thumbnailUrl,photos/35/caption,photos/35/pictureUrl,photos/35/thumbnailUrl,photos/36/caption,photos/36/pictureUrl,photos/36/thumbnailUrl,photos/37/caption,photos/37/pictureUrl,photos/37/thumbnailUrl,photos/38/caption,photos/38/pictureUrl,photos/38/thumbnailUrl,photos/39/caption,photos/39/pictureUrl,photos/39/thumbnailUrl,photos/40/caption,photos/40/pictureUrl,photos/40/thumbnailUrl,photos/41/caption,photos/41/pictureUrl,photos/41/thumbnailUrl,photos

### Step 01 - discarding information that doesn't matter

Removing columns relating to:
- `additionalHosts`
- property `photos` (we'll keep just first) 
- URL's
- `primaryHost/languages` (we'll keep just first) 

In [11]:
mask_additionalHosts = "df.columns.str.startswith('additionalHosts')"
mask_photos = "df.columns.str.contains(r'^photos/[^0]')"
mask_url = "df.columns.str.contains(r'^(?!photos).*Url$')"
mask_phlang = "df.columns.str.contains(r'^primaryHost/languages/[^0]')"
masks_commands = [mask_additionalHosts, mask_photos, mask_url, mask_phlang]

df_s01 = df_raw.pipe(remove_cols, masks_commands)

print(f'No Columns before step 01: {df_raw.shape[1]}')
print(f'No Columns after step 01: {df_s01.shape[1]}', end='\n\n')
df_s01.sample(1)

No Columns before step 01: 1107
No Columns after step 01: 452



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/about,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,reviews/0/author/firstName,reviews/0/author/hasProfilePic,reviews/0/author/id,reviews/0/author/smartName,reviews/0/collectionTag,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/language,reviews/0/localizedDate,reviews/0/localizedReview,reviews/0/localizedReview/comments,reviews/0/localizedReview/disclaimer,reviews/0/localizedReview/needsTranslation,reviews/0/localizedReview/response,reviews/0/rating,reviews/0/recipient/firstName,reviews/0/recipient/hasProfilePic,reviews/0/recipient/id,reviews/0/recipient/smartName,reviews/0/response,reviews/1/author/firstName,reviews/1/author/hasProfilePic,reviews/1/author/id,reviews/1/author/smartName,reviews/1/collectionTag,reviews/1/comments,reviews/1/createdAt,reviews/1/id,reviews/1/language,reviews/1/localizedDate,reviews/1/localizedReview,reviews/1/localizedReview/comments,reviews/1/localizedReview/disclaimer,reviews/1/localizedReview/needsTranslation,reviews/1/localizedReview/response,reviews/1/rating,reviews/1/recipient/firstName,reviews/1/recipient/hasProfilePic,reviews/1/recipient/id,reviews/1/recipient/smartName,reviews/1/response,reviews/2/author/firstName,reviews/2/author/hasProfilePic,reviews/2/author/id,reviews/2/author/smartName,reviews/2/collectionTag,reviews/2/comments,reviews/2/createdAt,reviews/2/id,reviews/2/language,reviews/2/localizedDate,reviews/2/localizedReview,reviews/2/localizedReview/comments,reviews/2/localizedReview/disclaimer,reviews/2/localizedReview/needsTranslation,reviews/2/localizedReview/response,reviews/2/rating,reviews/2/recipient/firstName,reviews/2/recipient/hasProfilePic,reviews/2/recipient/id,reviews/2/recipient/smartName,reviews/2/response,reviews/3/author/firstName,reviews/3/author/hasProfilePic,reviews/3/author/id,reviews/3/author/smartName,reviews/3/collectionTag,reviews/3/comments,reviews/3/createdAt,reviews/3/id,reviews/3/language,reviews/3/localizedDate,reviews/3/localizedReview,reviews/3/localizedReview/comments,reviews/3/localizedReview/disclaimer,reviews/3/localizedReview/needsTranslation,reviews/3/localizedReview/response,reviews/3/rating,reviews/3/recipient/firstName,reviews/3/recipient/hasProfilePic,reviews/3/recipient/id,reviews/3/recipient/smartName,reviews/3/response,reviews/4/author/firstName,reviews/4/author/hasProfilePic,reviews/4/author/id,reviews/4/author/smartName,reviews/4/collectionTag,reviews/4/comments,reviews/4/createdAt,reviews/4/id,reviews/4/language,reviews/4/localizedDate,reviews/4/localizedReview,reviews/4/localizedReview/comments,reviews/4/localizedReview/disclaimer,reviews/4/localizedReview/needsTranslation,reviews/4/localizedReview/response,reviews/4/rating,reviews/4/recipient/firstName,reviews/4/recipient/hasProfilePic,reviews/4/recipient/id,reviews/4/recipient/smartName,reviews/4/response,reviews/5/author/firstName,reviews/5/author/hasProfilePic,reviews/5/author/id,reviews/5/author/smartName,reviews/5/collectionTag,reviews/5/comments,reviews/5/createdAt,reviews/5/id,reviews/5/language,reviews/5/localizedDate,reviews/5/localizedReview,reviews/5/localizedReview/comments,reviews/5/localizedReview/disclaimer,reviews/5/localizedReview/needsTranslation,reviews/5/localizedReview/response,reviews/5/rating,reviews/5/recipient/firstName,reviews/5/recipient/hasProfilePic,reviews/5/recipient/id,reviews/5/recipient/smartName,reviews/5/response,reviews/6/author/firstName,reviews/6/author/hasProfilePic,reviews/6/author/id,reviews/6/author/smart

### Step 02 - Evaluate Missing Values

In [12]:
missing_values_table(df_s01)

Your selected dataframe has 452 columns.
There are 429 columns that have missing values.


,Missing Values,% of Total Values
reviews/2/collectionTag,547,100.000
reviews/15/collectionTag,547,100.000
reviews/9/localizedReview,547,100.000
reviews/1/localizedReview,547,100.000
reviews/9/collectionTag,547,100.000
reviews/14/collectionTag,547,100.000
reviews/8/localizedReview,547,100.000
reviews/18/localizedReview/response,547,100.000
reviews/14/localizedReview,547,100.000
reviews/18/localizedReview,547,100.000


### Step 03 - discarding information based on missing values

keeping only the following columns about reviews:
- Review: `comments`, `createdAt`, `id`, `rating`
- Review Author: `firstName`, `id`

In [13]:
mask_reviews_data = "df.columns.str.contains(r'^reviews/[0-9]+/(?!author/)(?!comments|createdAt|id|rating)')"
mask_author_data = "df.columns.str.contains(r'^reviews/[0-9]+/author/(?!firstName|id)')"
masks_commands = [mask_reviews_data, mask_author_data]

df_s03 = df_s01.pipe(remove_cols, masks_commands)

print(f'No Columns before step 02: {df_s01.shape[1]}')
print(f'No Columns after step 02: {df_s03.shape[1]}', end='\n\n')
df_s03.sample(1)

No Columns before step 02: 452
No Columns after step 02: 152



,address,isAvailable,isHostedBySuperhost,location/lat,location/lng,name,numberOfGuests,photos/0/caption,photos/0/pictureUrl,photos/0/thumbnailUrl,pricing/rate/amount,pricing/rate/amountFormatted,pricing/rate/currency,pricing/rate/isMicrosAccuracy,pricing/rateType,primaryHost/about,primaryHost/badges/0,primaryHost/badges/1,primaryHost/badges/2,primaryHost/firstName,primaryHost/hasInclusionBadge,primaryHost/hasProfilePic,primaryHost/id,primaryHost/isSuperHost,primaryHost/languages/0,primaryHost/memberSince,primaryHost/responseRate,primaryHost/responseTime,primaryHost/smartName,reviews/0/author/firstName,reviews/0/author/id,reviews/0/comments,reviews/0/createdAt,reviews/0/id,reviews/0/rating,reviews/1/author/firstName,reviews/1/author/id,reviews/1/comments,reviews/1/createdAt,reviews/1/id,reviews/1/rating,reviews/2/author/firstName,reviews/2/author/id,reviews/2/comments,reviews/2/createdAt,reviews/2/id,reviews/2/rating,reviews/3/author/firstName,reviews/3/author/id,reviews/3/comments,reviews/3/createdAt,reviews/3/id,reviews/3/rating,reviews/4/author/firstName,reviews/4/author/id,reviews/4/comments,reviews/4/createdAt,reviews/4/id,reviews/4/rating,reviews/5/author/firstName,reviews/5/author/id,reviews/5/comments,reviews/5/createdAt,reviews/5/id,reviews/5/rating,reviews/6/author/firstName,reviews/6/author/id,reviews/6/comments,reviews/6/createdAt,reviews/6/id,reviews/6/rating,reviews/7/author/firstName,reviews/7/author/id,reviews/7/comments,reviews/7/createdAt,reviews/7/id,reviews/7/rating,reviews/8/author/firstName,reviews/8/author/id,reviews/8/comments,reviews/8/createdAt,reviews/8/id,reviews/8/rating,reviews/9/author/firstName,reviews/9/author/id,reviews/9/comments,reviews/9/createdAt,reviews/9/id,reviews/9/rating,reviews/10/author/firstName,reviews/10/author/id,reviews/10/comments,reviews/10/createdAt,reviews/10/id,reviews/10/rating,reviews/11/author/firstName,reviews/11/author/id,reviews/11/comments,reviews/11/createdAt,reviews/11/id,reviews/11/rating,reviews/12/author/firstName,reviews/12/author/id,reviews/12/comments,reviews/12/createdAt,reviews/12/id,reviews/12/rating,reviews/13/author/firstName,reviews/13/author/id,reviews/13/comments,reviews/13/createdAt,reviews/13/id,reviews/13/rating,reviews/14/author/firstName,reviews/14/author/id,reviews/14/comments,reviews/14/createdAt,reviews/14/id,reviews/14/rating,reviews/15/author/firstName,reviews/15/author/id,reviews/15/comments,reviews/15/createdAt,reviews/15/id,reviews/15/rating,reviews/16/author/firstName,reviews/16/author/id,reviews/16/comments,reviews/16/createdAt,reviews/16/id,reviews/16/rating,reviews/17/author/firstName,reviews/17/author/id,reviews/17/comments,reviews/17/createdAt,reviews/17/id,reviews/17/rating,reviews/18/author/firstName,reviews/18/author/id,reviews/18/comments,reviews/18/createdAt,reviews/18/id,reviews/18/rating,reviews/19/author/firstName,reviews/19/author/id,reviews/19/comments,reviews/19/createdAt,reviews/19/id,reviews/19/rating,roomType,stars,url
434,"São Paulo, Brasil",true,false,-23.62425,-46.67702,"Impecavel, localização excelente",2,Living Comedor,https://a0.muscache.com/im/pictures/f3b32b0d-2b3c-4984-b2f2-02c979ec2ad5.jpg?aki_policy=large,https://a0.muscache.com/im/pictures/f3b32b0d-2b3c-4984-b2f2-02c979ec2ad5.jpg?aki_policy=small,1200,R$1.200,BRL,false,nightly,NaN,5 avaliações,Identidade verificada,NaN,Bettina,false,true,31878709,false,English,Membro desde abril de 2015,100%,dentro de um dia,Bettina,Fabio,27836142,"O apartamento e muito legal, arrumado , varanda bacana , mobilia bonita , piscina gostosa , gostei muito estendi minha estadia por muito mais tempo , tudo completo",2015-12-01T14:51:10Z,55397518,5,Fabio,27836142,"Apartamento muito arrumado e bonito , gostei muito da varanda , possui todos os utensílios de cozinha e o prédio possui um ótimo astral com moradores novos mas definitivamente o melhor de tudo 'e a piscina na cobertura . Quanto a Betina ela foi muito amável e atenciosa , recomendo , inclusive pretendo extender minha esta

### Step 04 - Getting text columns from dataframe

In [14]:
# Definindo as  colunas de texto
mask_text_cols = "df.columns.str.contains(r'name|about|comments')"
df_txt_cols = df_s03.pipe(remove_cols, [mask_text_cols], inverse=False)
df_txt_cols.sample(1)

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments,reviews/10/comments,reviews/11/comments,reviews/12/comments,reviews/13/comments,reviews/14/comments,reviews/15/comments,reviews/16/comments,reviews/17/comments,reviews/18/comments,reviews/19/comments
27,Loft República/Viera Carvalho (ad),NaN,Ademar foi super solícito e me ajudou em tudo que foi necessário. Extremamente simpático e gentil.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
arr_texts = df_txt_cols.fillna('').values
arr_texts[:5]

array([['Ap 2q conforto e comodidade!', '',
        'The host canceled this reservation 107 days before arrival. This is an automated posting.',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', '', ''],
       ['Apartamento Guarujá Praia Enseada Sp (GRUPO REIS)', '', '', '',
        '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
        '', ''],
       ['Casa na Praia de Pernambuco, Guarujá - SP', '',
        'A casa fica bem próxima da praia, além disso a casa é uma graça, tudo muito limpo e organizado. Os anfritriões são muito atenciosos. Recomendo, voltaremos mais vezes!',
        'Um lugar lindo e próximo a praia, da para ir de a pé.  Cristian e dona Maria super atenciosos e cuidadosos conosco. Casa super limpa e com utensílios que dá para cozinhar sem problemas. A praia é de água limpa e bem familiar. Amamos o lugar e pretendemos voltar mais vezes.  Valeu muito a pena !',
        'Cristian e Dona Maria excelentes anfitriões! <br/

### Step 05 - Defining preprocessing and cleaning operations

In [16]:
if LANG == 'english':
  stanza_lang = 'en'
  spacy_pipe = 'en_core_web_lg'
  
elif LANG == 'portuguese':
  stanza_lang = 'pt'
  spacy_pipe = 'pt_core_news_lg'
  
else:
  print('Antes de continuar, defina a variavel LANG = "portuguese" | "english"')

In [17]:
from urllib.request import Request, urlopen 

req = Request(f'https://countwordsfree.com/stopwords/{LANG}/txt')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)
stop_words = pd.read_csv(content, header=None)
stop_words = stop_words[0].values

In [18]:
!python -m spacy download {spacy_pipe}


2023-04-13 14:28:18.518980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 14:28:19.727640: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-13 14:28:21.247541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-13 14:28:21.247990: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [19]:
from functools import cache
from typing import Union
import spacy
spacy.prefer_gpu()
import stanza


stanza.download(stanza_lang)
nlp_spacy = spacy.load(spacy_pipe, disable=['parser', 'ner', 'textcat']) 
spacy_type = type(nlp_spacy)

@cache
def preprocess_txt(textraw:str, nlp_obj:Union[spacy_type, stanza.pipeline.core.Pipeline], lemma=False) -> str:
  '''
  Realiza a limpeza e o pre processamento de um dado texto em portugues.
  As etapas de pré-processamento a serem executadas são:
  * Conversão para Minúsculas.
  * Remocao de tags HTML.
  * Remocao de pontuacoes.
  * Remocao de algarismos numericos e numeros de telefone.
  * Remocao de multiplos espacos.
  * Remocao de URLs e E-mails: são removidos, pois eles não adicionam significado ao texto.
  * Substituicao de Emojis: Substitui emojis usando um dicionário predefinido contendo emojis junto com seu significado. (por exemplo: ":)" para "sorriso")
  * Substituindo nomes de usuário: Substitua @Usernames pela palavra "USER". (por exemplo: "@nick_name" para "USER")
  * Remocao de ascentuacao.
  * Removendo Não-Alfabetos: Substitui caracteres (exceto Dígitos e Alfabetos) por um espaço.
  * Removendo letras consecutivas: 3 ou mais letras consecutivas são substituídas por 2 letras. (por exemplo: "Muitoooo" para "Muitoo")
  * Remoção de palavras curtas: palavras com comprimento menor que 2 são removidas.
  * Removendo Stopwords: Stopwords são as palavras que não adicionam muito significado a uma frase. Eles podem ser ignorados com segurança sem sacrificar o significado da frase. (por exemplo: "the", "he", "have")
  * Lematização: A lematização é o processo de converter uma palavra em sua forma básica. (por exemplo: “Great” para “Good”). A lematização geralmente retorna palavras válidas (que existem) enquanto as técnicas de stemming retornam (na maioria das vezes) palavras abreviadas

  Parameters:
    textraw (str): texto bruto o qual sofrera as operacoes de limpeza e pre processamento.
    nlp_obj (spacy.lang.pt.Portuguese | spacy.lang.en.English | stanza.pipeline.core.Pipeline): - objeto que indica a biblioteca a ser utilizada para a etapa de tokenizacao, remocao de stop words e lemmatizacao.
    lemma (boolean): (opcional) se deve substituir as palavras originais do texto por seu lemma ou nao. Default: False

  Returns:
    wordsclean (pd.Dataframe): dataframe sem as colunas indicadas na máscara
  '''
  import re
  from unidecode import unidecode
  # from gensim.utils import simple_preprocess
  
  global stop_words

  # Definindo padrões Regex
  # urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
  # userPattern       = '@[^\s]+'
  alphaPattern      = "[^a-zA-Z0-9]"
  sequencePattern   = r"(.)\1\1+" # letras consecutivas
  seqReplacePattern = r"\1\1"
  
  # Definindo dicionario de emojis e seus respectivos significados
  emojis = {':)': 'sorri', ':-)': 'sorri', ';d': 'pisca', ':-E': 'vampiro', ':(': 'triste', 
            ':-(': 'triste', ':-<': 'triste', ':P': 'mostra lingua', ':O': 'surpresa',
            ':-@': 'chocado', ':@': 'chocado',':-$': 'confuso', ':\\': 'irritado', 
            ':#': 'mudo', ':X': 'mudo', ':^)': 'sorri', ':-&': 'confuso', '$_$': 'ganancia',
            '@@': 'vira os olhos', ':-!': 'confuso', ':-D': 'sorri', ':-0': 'grita', 'O.o': 'confuso',
            '<(-_-)>': 'robo', 'd[-_-]b': 'dj', ":'-)": 'sorri triste', ';)': 'piscar', 
            ';-)': 'piscar', 'O:-)': 'anjo','O*-)': 'anjo','(:-D': 'fofoca', '=^.^=': 'gato'}

   
  # ##################
  # Tentativa de aplicação de técnica para correção de palavras com erro de escrita, 
  # mas o resultado não foi considerado satisfatório.
  # ##################
  # from gensim.utils import simple_tokenize
  # from spellchecker import SpellChecker
  # spell = SpellChecker(language='pt')
  # wordlist = list(simple_tokenize(textraw))
  # misspelled = spell.unknown(wordlist)
  # correctedtext = [spell.correction(word) for word in misspelled]
  # print(correctedtext)

  textclean = neat_preprocessing(textraw)  
  for emoji in emojis.keys():
      textclean = textclean.replace(emoji, "EMOJI" + emojis[emoji])  
  textclean = re.sub(sequencePattern, seqReplacePattern, textclean)
  textclean = unidecode(textclean)
  textclean = re.sub(alphaPattern, " ", textclean)
  wordsclean = ''

  if nlp_obj.__module__ == 'spacy.lang.en' or nlp_obj.__module__ == 'spacy.lang.pt':
    doc = nlp_obj(textclean)
    for token in doc:      
      if not token.is_stop and len(token) > 1:
        if lemma and token.pos_ == 'VERB':
          wordsclean += token.lemma_+' '
        else:
          wordsclean += token.text + ' '

  elif isinstance(nlp_obj, stanza.pipeline.core.Pipeline):
    doc = nlp_obj(textclean)
    for sent in doc.sentences:
      for word in sent.words:   
        if word.text not in stop_words and len(word.text) > 1:   
          if lemma and word.upos == 'VERB':
            wordsclean += word.lemma +' '
          else:
            wordsclean += word.text + ' '
  else:
      raise BaseException('Instancia nao reconhecida do objeto "nlp_obj". Opcoes: spacy | stanza')
    
  return wordsclean


INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


### Step 06 - Tests

In [20]:
arr_texts[133]

array(['Casa de Veraneio na Praia de Pernambuco', '',
       'A casa é perfeita para reunir a família com segurança e sossego. <br/>São cinco suítes que oferecem privacidade para todos os hóspedes. <br/>Reunimos 13 familiares  num ambiente gostoso com uma piscina maravilhosa e limpa, inclusive com som  ambiente. <br/>O ar condicionado é muito eficiente em todos os quartos. A Edna cozinha divinamente, e o Mauro deixa as dependências imaculadas. Ela nos fez uma lula à dorè dos deuses ! Super crocantes ! A caipirinha do Mauro é digno de um  barman!<br/>Recomendo sem restrições!!!',
       'Mesmo sem conhecer pessoalmente a Carolina ela sempre foi muito educada, simpática e rápida/eficiente nas respostas. A casa superou todas as nossas expectativas, o local é enorme e acomodou confortavelmente as 14 pessoas que viajaram comigo. Os caseiros ficaram todo o tempo na casa e nos ajudaram com algumas tarefas do dia a dia, com destaque para o Feijão da Edna e o Churrasco do Mauro. Está nos meus p

#### Teste Spacy

In [21]:
%%time

postprocess = []
for rawtxt in arr_texts[133]:
  if len(rawtxt):
    postprocess.append(preprocess_txt(rawtxt, nlp_obj=nlp_spacy, lemma=True))

postprocess

CPU times: user 608 ms, sys: 205 ms, total: 813 ms
Wall time: 845 ms


['casa de veraneio na praia de pernambuco ',
 'casa perfeita para reunir familia com seguranca sossego sao cinco suites que oferecem privacidade para todos os hospedes reunimos familiares num ambiente gostoso com uma piscina maravilhosa limpa inclusive com som ambiente ar condicionado muito eficiente em todos os quartos edna cozinha divinamente mauro deixa dependencias imaculadas ela nos fez uma lula dore dos deuse super crocantes caipirinha mauro digno de um barmanrecomendo sem restricoes ',
 'mesmo sem conhecer pessoalmente carolina ela sempre foi muito educada simpatica rapida eficiente nas resposta casa superou todas nossas expectativas local enorme acomodou confortavelmente pessoas que viajaram comigo os caseiros ficaram todo tempo na casa nos ajudaram com algumas tarefas dia dia com destaque para feijao da edna churrasco mauro esta nos meus planos voltar em breve para passar mais alguns dias ',
 'carolina home incredible house property beautiful person pictures house pool perfect

####  Teste Stanza

In [22]:
%%time
########## Stanza defined's processors for this task:
# tokenize - Segments a Document into Sentences, each containing a list of Tokens.
# mwt - Expands multi-word tokens (MWTs) into multiple words when they are predicted by the tokenizer.Fix contractions.
# pos - UPOS, XPOS, and UFeats annotations are accessible through Word’s properties `pos`, `xpos`, and `ufeats`.
# lemma - Perform lemmatization on a Word using the `Word.text` and `Word.upos` values. The result can be accessed as `Word.lemma`. 
nlp_stanza = stanza.Pipeline(lang=stanza_lang, processors='tokenize,mwt,pos,lemma', use_gpu=True, download_method=None, verbose=False, max_cache_size=2)
postprocess = []
for rawtxt in arr_texts[133]:
  if len(rawtxt):
    postprocess.append(preprocess_txt(rawtxt, nlp_obj=nlp_stanza, lemma=True))

postprocess

CPU times: user 1.1 s, sys: 234 ms, total: 1.34 s
Wall time: 1.79 s


['casa veraneio praia pernambuco ',
 'casa perfeita para reunir familia seguranca sossego sao cinco suites oferecem privacidade para todos os hospedes reunimos familiares num ambiente gostoso uma piscina maravilhosa limpa inclusive som ambiente ar condicionado muito eficiente em todos os quartos edna cozinha divinamente mauro deixa dependencias imaculadas ela fez uma lula dore dos deuses super crocantes caipirinha mauro digno um barmanrecomendo sem restricoes ',
 'mesmo sem conhecer pessoalmente carolina ela sempre foi muito educada simpatica rapida eficiente nas respostas casa superou todas nossas expectativas local enorme acomodou confortavelmente pessoas viajaram comigo os caseiros ficaram todo tempo casa ajudaram algumas tarefas dia dia destaque para feijao da edna churrasco mauro esta meus planos voltar em breve para passar mais alguns dias ',
 'carolina incredible house property beautiful person pictures house pool perfect multigenerational family art lovers multiple lounge game 

### Step 07 - Applying preprocessing operations on dataframe text columns

#### Using SpaCy

In [24]:
%%time
df_txt_cols_processed_spacy = parallel_applymap(df_txt_cols, preprocess_txt, worker_count=10, na_action='ignore', nlp_obj=nlp_spacy, lemma=True)

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

...

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

CPU times: user 1min 40s, sys: 9.3 s, total: 1min 50s
Wall time: 1min 55s


In [25]:
df_txt_cols_processed_spacy.loc[133:134]

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments,reviews/10/comments,reviews/11/comments,reviews/12/comments,reviews/13/comments,reviews/14/comments,reviews/15/comments,reviews/16/comments,reviews/17/comments,reviews/18/comments,reviews/19/comments
133,casa de veraneio na praia de pernambuco,NaN,casa perfeita para reunir familia com seguranca sossego sao cinco suites que oferecem privacidade para todos os hospedes reunimos familiares num ambiente gostoso com uma piscina maravilhosa limpa inclusive com som ambiente ar condicionado muito eficiente em todos os quartos edna cozinha divinamente mauro deixa dependencias imaculadas ela nos fez uma lula dore dos deuse super crocantes caipirinha mauro digno de um barmanrecomendo sem restricoes,mesmo sem conhecer pessoalmente carolina ela sempre foi muito educada simpatica rapida eficiente nas resposta casa superou todas nossas expectativas local enorme acomodou confortavelmente pessoas que viajaram comigo os caseiros ficaram todo tempo na casa nos ajudaram com algumas tarefas dia dia com destaque para feijao da edna churrasco mauro esta nos meus planos voltar em breve para passar mais alguns dias,carolina home incredible house property beautiful person pictures house pool perfect multigenerational family art lovers multiple lounge game play areas beach blocks away stunning beaches visit short drive away understand work caseros bit learning curve speak portugues accustomed have day figure mauro edna fantastic thing improve wifi extend entire house highly recommend carolina property hope visit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,alto padrao jardins,engenheiro empresario casado filhos,gil apartment fantastic location great apartment spacious cozy gil helpful kind recommend place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Using Stanza

In [26]:
%%time
df_txt_cols_processed_stanza = parallel_applymap(df_txt_cols, preprocess_txt, worker_count=10, na_action='ignore', nlp_obj=nlp_stanza, lemma=True)

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

...

  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/1210 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/1188 [00:00<?, ?it/s]

CPU times: user 12min 12s, sys: 1min 23s, total: 13min 35s
Wall time: 10min 25s


In [27]:
df_txt_cols_processed_stanza.loc[133:134]

,name,primaryHost/about,reviews/0/comments,reviews/1/comments,reviews/2/comments,reviews/3/comments,reviews/4/comments,reviews/5/comments,reviews/6/comments,reviews/7/comments,reviews/8/comments,reviews/9/comments,reviews/10/comments,reviews/11/comments,reviews/12/comments,reviews/13/comments,reviews/14/comments,reviews/15/comments,reviews/16/comments,reviews/17/comments,reviews/18/comments,reviews/19/comments
133,casa veraneio praia pernambuco,NaN,casa perfeita para reunir familia seguranca sossego sao cinco suites oferecem privacidade para todos os hospedes reunimos familiares num ambiente gostoso uma piscina maravilhosa limpa inclusive som ambiente ar condicionado muito eficiente em todos os quartos edna cozinha divinamente mauro deixa dependencias imaculadas ela fez uma lula dore dos deuses super crocantes caipirinha mauro digno um barmanrecomendo sem restricoes,mesmo sem conhecer pessoalmente carolina ela sempre foi muito educada simpatica rapida eficiente nas respostas casa superou todas nossas expectativas local enorme acomodou confortavelmente pessoas viajaram comigo os caseiros ficaram todo tempo casa ajudaram algumas tarefas dia dia destaque para feijao da edna churrasco mauro esta meus planos voltar em breve para passar mais alguns dias,carolina incredible house property beautiful person pictures house pool perfect multigenerational family art lovers multiple lounge game play areas beach blocks stunning beaches visit short drive understand work caseros bit learning curve speak portugues accustomed day figure mauro edna fantastic improve wifi extend entire house highly recommend carolina property hope visit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,alto padrao jardins,engenheiro empresario casado filhos,gil apartment fantastic location great apartment spacious cozy gil helpful kind recommend place,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
